In [6]:
import numpy as np
import random
import tsplib95

# Load TSP data from a file
def load_tsp_data(filename):
    problem = tsplib95.load(filename)
    return problem

# Create a random route
def create_route(problem):
    # Adjust for 1-based indexing used in TSPLIB
    return np.random.permutation(range(1, problem.dimension + 1))

# Initialize a population of routes
def initialize_population(pop_size, problem):
    return [create_route(problem) for _ in range(pop_size)]

# Calculate the total distance of a route
def calculate_fitness(route, problem):
    # Wrap around the route to form a complete loop
    wrapped_route = np.append(route, route[0])
    return sum(problem.get_weight(wrapped_route[i], wrapped_route[i + 1]) for i in range(len(route)))

# Selection: Tournament Selection
def tournament_selection(population, problem, k=3):
    selected = []
    for _ in range(len(population)):
        contenders = random.sample(population, k)
        best = min(contenders, key=lambda route: calculate_fitness(route, problem))
        selected.append(best)
    return selected

# Crossover: Ordered Crossover
def ordered_crossover(parent1, parent2):
    size = len(parent1)
    child = [-1] * size
    start, end = sorted(random.sample(range(size), 2))

    # Copy a slice from first parent to child
    for i in range(start, end + 1):
        child[i] = parent1[i]

    # Fill the child with the elements from the second parent in the order they appear
    p2_elements = [item for item in parent2 if item not in child]
    child_idx, p2_idx = 0, 0
    while -1 in child:
        if child[child_idx] == -1:
            child[child_idx] = p2_elements[p2_idx]
            p2_idx += 1
        child_idx += 1

    return child

# Mutation: Swap Mutation
def swap_mutation(route, mutation_rate):
    for i in range(len(route)):
        if random.random() < mutation_rate:
            swap_idx = random.randint(0, len(route) - 1)
            route[i], route[swap_idx] = route[swap_idx], route[i]
    return route
        
def genetic_algorithm(problem, pop_size, generations, mutation_rate):
    population = initialize_population(pop_size, problem)
    for _ in range(generations):
        # Selection
        selected = tournament_selection(population, problem)

        # Crossover
        children = []
        for i in range(0, pop_size, 2):
            if i + 1 < len(selected):
                child1 = ordered_crossover(selected[i], selected[i+1])
                child2 = ordered_crossover(selected[i+1], selected[i])
                children.append(swap_mutation(child1, mutation_rate))
                children.append(swap_mutation(child2, mutation_rate))

        population = children

    # Select the best solution
    best_route = min(population, key=lambda route: calculate_fitness(route, problem))
    return best_route, calculate_fitness(best_route, problem)

# Example execution
problem = load_tsp_data("/Users/mwr/Downloads/NEC_A4/dantzig42.tsp.txt")
best_route, best_distance = genetic_algorithm(problem, pop_size=50, generations=100, mutation_rate=0.01)
print(f"Best Route: {best_route}\nTotal Distance: {best_distance}")

Best Route: [14, 15, 16, 4, 1, 42, 41, 2, 40, 5, 38, 39, 37, 26, 25, 27, 22, 17, 18, 19, 12, 10, 8, 3, 9, 31, 30, 29, 32, 33, 34, 35, 36, 6, 7, 23, 20, 21, 28, 24, 11, 13]
Total Distance: 1222
